In [1]:
import sys
sys.path.append("..")

In [2]:
from keras.models import Model

Using TensorFlow backend.


In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [5]:
import bowizer
import tfidf

In [6]:
with open('../data/norvig/big.txt') as f:
    norvig = f.read()

In [7]:
chunks = make_chunks(norvig)

In [8]:
r = np.random.RandomState(42)

In [9]:
r.shuffle(chunks)
train_text, test_text = make_train_test(chunks)

In [10]:
tm = bowizer.TokenMaker([norvig], 3000)

In [11]:
embed_h = EmbeddingHyper(tm.vocab_size + 1, 256)
conv_h = ConvHyper(256, 6, 4)
rnn_h = RnnHyper(512, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [12]:
dernn_h = RnnHyper(512, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(256, 6, 4)
decoder_h = td.Hyper(tm.vocab_size + 1, [dernn_h, dec_h])

In [13]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [14]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [15]:
model64, model128, model256 = make_model(64), make_model(128), make_model(256)

In [16]:
models = {64: model64, 128: model128, 256: model256 }

In [17]:
def training_round(max_len, history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train, test = bowizer.SlicedWordData.Random(train_text, test_text, 2000, max_len, 10000, r, tm)
    model = models[max_len]
    newhistory = model.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=16,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [18]:
history = None

In [19]:
for i in range(20):
    print(i)
    history = training_round(64, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 145s - loss: 5.0054 - categorical_accuracy: 0.2152 - val_loss: 4.5686 - val_categorical_accuracy: 0.2679
Epoch 2/5
10000/10000 [==============================] - 142s - loss: 4.6941 - categorical_accuracy: 0.2185 - val_loss: 4.3911 - val_categorical_accuracy: 0.2685
Epoch 3/5
10000/10000 [==============================] - 146s - loss: 4.4900 - categorical_accuracy: 0.2223 - val_loss: 4.2173 - val_categorical_accuracy: 0.2739
Epoch 4/5
10000/10000 [==============================] - 147s - loss: 4.2881 - categorical_accuracy: 0.2305 - val_loss: 4.0572 - val_categorical_accuracy: 0.2827
Epoch 5/5
10000/10000 [==============================] - 147s - loss: 4.0970 - categorical_accuracy: 0.2415 - val_loss: 3.9489 - val_categorical_accuracy: 0.2913
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 146s - loss: 4.0306 - categori

10000/10000 [==============================] - 151s - loss: 0.9989 - categorical_accuracy: 0.7444 - val_loss: 1.3065 - val_categorical_accuracy: 0.6954
Epoch 48/50
10000/10000 [==============================] - 148s - loss: 0.9117 - categorical_accuracy: 0.7621 - val_loss: 1.3327 - val_categorical_accuracy: 0.6930
Epoch 49/50
10000/10000 [==============================] - 147s - loss: 0.8510 - categorical_accuracy: 0.7751 - val_loss: 1.3411 - val_categorical_accuracy: 0.6947
Epoch 50/50
10000/10000 [==============================] - 153s - loss: 0.7980 - categorical_accuracy: 0.7868 - val_loss: 1.3576 - val_categorical_accuracy: 0.6947
10
Train on 10000 samples, validate on 1000 samples
Epoch 51/55
10000/10000 [==============================] - 151s - loss: 1.1076 - categorical_accuracy: 0.7231 - val_loss: 1.1638 - val_categorical_accuracy: 0.7215
Epoch 52/55
10000/10000 [==============================] - 154s - loss: 0.9118 - categorical_accuracy: 0.7649 - val_loss: 1.1796 - val_categ

10000/10000 [==============================] - 146s - loss: 0.4190 - categorical_accuracy: 0.8945 - val_loss: 0.7597 - val_categorical_accuracy: 0.8305
Epoch 95/95
10000/10000 [==============================] - 146s - loss: 0.3990 - categorical_accuracy: 0.8987 - val_loss: 0.7742 - val_categorical_accuracy: 0.8298
19
Train on 10000 samples, validate on 1000 samples
Epoch 96/100
10000/10000 [==============================] - 148s - loss: 0.5915 - categorical_accuracy: 0.8551 - val_loss: 0.6835 - val_categorical_accuracy: 0.8405
Epoch 97/100
10000/10000 [==============================] - 146s - loss: 0.4472 - categorical_accuracy: 0.8903 - val_loss: 0.6956 - val_categorical_accuracy: 0.8412
Epoch 98/100
10000/10000 [==============================] - 146s - loss: 0.4136 - categorical_accuracy: 0.8975 - val_loss: 0.7024 - val_categorical_accuracy: 0.8404
Epoch 99/100
10000/10000 [==============================] - 146s - loss: 0.3975 - categorical_accuracy: 0.9011 - val_loss: 0.7461 - val_c

In [20]:
history = None
for i in range(20):
    print(i)
    history = training_round(128, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 251s - loss: 4.1157 - categorical_accuracy: 0.2910 - val_loss: 3.6453 - val_categorical_accuracy: 0.3464
Epoch 2/5
10000/10000 [==============================] - 247s - loss: 3.4233 - categorical_accuracy: 0.3368 - val_loss: 3.1830 - val_categorical_accuracy: 0.3896
Epoch 3/5
10000/10000 [==============================] - 246s - loss: 3.0112 - categorical_accuracy: 0.3886 - val_loss: 2.9223 - val_categorical_accuracy: 0.4312
Epoch 4/5
10000/10000 [==============================] - 246s - loss: 2.7527 - categorical_accuracy: 0.4317 - val_loss: 2.7381 - val_categorical_accuracy: 0.4600
Epoch 5/5
10000/10000 [==============================] - 246s - loss: 2.9271 - categorical_accuracy: 0.4052 - val_loss: 2.8879 - val_categorical_accuracy: 0.4287
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 250s - loss: 2.6108 - categori

KeyboardInterrupt: 

In [ ]:
history = None
for i in range(20):
    print(i)
    history = training_round(64, history)